In [1]:
# @title Setup

competition = "Level 2: liver-fibrosis"  # @param
# @markdown ---

from google.colab import userdata
import json

# Get the Kaggle credentials from Colab's userdata
username = userdata.get("KAGGLE_USER")
key = userdata.get("KAGGLE_KEY")

# Echo the credentials into the kaggle.json file
!mkdir -p ~/.kaggle
!echo '{{"username":"{username}","key":"{key}"}}' > ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

competition_name = "liver-fibrosis-severity-prediction"

! kaggle competitions download -c {competition_name}
! unzip /content/{competition_name}.zip
! rm -rf {competition_name}.zip

Streaming output truncated to the last 5000 lines.
  inflating: liver-fibrosis-severity-prediction/liver-lesion/images/53584.jpg  
  inflating: liver-fibrosis-severity-prediction/liver-lesion/images/53595.jpg  
  inflating: liver-fibrosis-severity-prediction/liver-lesion/images/5362.jpg  
  inflating: liver-fibrosis-severity-prediction/liver-lesion/images/53629.jpg  
  inflating: liver-fibrosis-severity-prediction/liver-lesion/images/5363.jpg  
  inflating: liver-fibrosis-severity-prediction/liver-lesion/images/53632.jpg  
  inflating: liver-fibrosis-severity-prediction/liver-lesion/images/53640.jpg  
  inflating: liver-fibrosis-severity-prediction/liver-lesion/images/53644.jpg  
  inflating: liver-fibrosis-severity-prediction/liver-lesion/images/53652.jpg  
  inflating: liver-fibrosis-severity-prediction/liver-lesion/images/5366.jpg  
  inflating: liver-fibrosis-severity-prediction/liver-lesion/images/53665.jpg  
  inflating: liver-fibrosis-severity-prediction/liver-lesion/images/5367

In [2]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import shutil
import glob
import os

In [ ]:
images_dir = '/content/liver-fibrosis-severity-prediction/liver-lesion/images/'
annotations_dir = '/content/liver-fibrosis-severity-prediction/liver-lesion/annotations/'

In [ ]:
try:
    !rm -rf /content/yolo_data
except:
    pass

os.makedirs('/content/yolo_data/images/train/', exist_ok=True)
os.makedirs('/content/yolo_data/labels/train/', exist_ok=True)

for file in tqdm(os.listdir(annotations_dir)):
    try:
        img_path = images_dir + file.split('.')[0] + '.jpg'
        shutil.copy(img_path, '/content/yolo_data/images/train/')

        annotation_path = annotations_dir + file
        shutil.copy(annotation_path, '/content/yolo_data/labels/train/')
    except:
        pass

100%|██████████| 7222/7222 [00:03<00:00, 2035.37it/s]


In [ ]:
yaml_content = """
path: /content/yolo_data/
train: images/train/
val: images/train/

nc: 7
names: ['FFC', 'FFS', 'HCC', 'cyst', 'hemangioma', 'dysplastic', 'CCA']
"""

with open('/content/yolo_data/data.yaml', 'w') as f:
    f.write(yaml_content)

print("Created data.yaml file.")

Created data.yaml file.


## YOLO

In [3]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.3 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')

results = model.train(data='/content/yolo_data/data.yaml', epochs=20, imgsz=640)
model.save('yolo11n_liver.pt')

Ultralytics 8.3.149 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretraine

train: Scanning /content/yolo_data/labels/train.cache... 7222 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7222/7222 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 434.0±153.2 MB/s, size: 456.8 KB)


val: Scanning /content/yolo_data/labels/train.cache... 7222 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7222/7222 [00:00<?, ?it/s]


...

                   all       7222       8709      0.791      0.745      0.807      0.455

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20      11.8G      1.511      1.083      1.279          6        640: 100%|██████████| 452/452 [01:10<00:00,  6.41it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 226/226 [00:36<00:00,  6.14it/s]


                   all       7222       8709      0.804      0.754      0.817      0.466

20 epochs completed in 0.607 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 5.5MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.149 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,583,517 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 226/226 [00:38<00:00,  5.93it/s]


                   all       7222       8709      0.807      0.753      0.817      0.466
                   FFC        858        962      0.827       0.81      0.875      0.525
                   FFS       1522       1806       0.76      0.722      0.795      0.398
                   HCC       1592       1740      0.896      0.852      0.918      0.583
                  cyst       1244       1358      0.866      0.812      0.878      0.538
            hemangioma       1330       1342      0.879      0.851      0.907      0.525
            dysplastic        587        626      0.816      0.773      0.833      0.438
                   CCA        187        875      0.602       0.45      0.511      0.255
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train3


## Inference

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
yolo_dir = '/content/drive/MyDrive/SAI5_Onsite/Hack05_Liver-fibrosis/test_YOLO_added/'

try:
    !rm -rf {yolo_dir}
except:
    pass

os.makedirs(yolo_dir, exist_ok=True)

In [19]:
!rm -rf yolo_data/
os.makedirs('yolo_data/', exist_ok=True)

shutil.copytree("/content/drive/MyDrive/SAI5_Onsite/Hack05_Liver-fibrosis/train/", "/content/yolo_data/train/",)
shutil.copytree("/content/drive/MyDrive/SAI5_Onsite/Hack05_Liver-fibrosis/test/", "/content/yolo_data/test/")

'/content/yolo_data/test/'

In [26]:
from ultralytics import YOLO

model = YOLO('/content/drive/MyDrive/SAI5_Onsite/Hack05_Liver-fibrosis/yolo11n_liver.pt')
inference_images = glob.glob('/content/yolo_data/test/*.png')

# store each image class in dict
class_dict = {}
for image in tqdm(inference_images):
    results = model(image, verbose=False, stream=False)

    file_id = image.split('/')[-1].replace('.png', '')
    results[0].save(yolo_dir + file_id + '.png')

    class_dict[file_id] = results[0].boxes.cls.cpu().numpy()


# change empty list -> nan
for key, val in class_dict.items():
    if len(val) == 0:
        class_dict[key] = np.nan

100%|██████████| 433/433 [00:10<00:00, 40.83it/s]


In [33]:
bounding_boxes_dict = {}
for image in tqdm(inference_images):
    results = model(image, verbose=False, stream=False)

    file_id = image.split('/')[-1].replace('.png', '')

    # Extract bounding box coordinates (xyxy format)
    bboxes = results[0].boxes.xyxy.cpu().numpy()

    confidences = results[0].boxes.conf.cpu().numpy()  # conf. score
    classes = results[0].boxes.cls.cpu().numpy()  # class indices

    bounding_boxes_dict[file_id] = {
        'boxes': bboxes,
        'confidences': confidences,
        'classes': classes
    }


data_list = []
for file_id, box_info in bounding_boxes_dict.items():
    boxes = box_info['boxes']
    confidences = box_info['confidences']
    classes = box_info['classes']

    if len(boxes) > 0:
        for i in range(len(boxes)):
            data_list.append({
                'image_name': file_id,
                'x1': boxes[i][0],
                'y1': boxes[i][1],
                'x2': boxes[i][2],
                'y2': boxes[i][3],
                'confidence': confidences[i],
                'class_id': int(classes[i])
            })
    else:
         data_list.append({
                'image_name': file_id,
                'x1': np.nan,
                'y1': np.nan,
                'x2': np.nan,
                'y2': np.nan,
                'confidence': np.nan,
                'class_id': np.nan
            })

df_boxes = pd.DataFrame(data_list)
print("\nBounding Box DataFrame:")
print(df_boxes.shape)
df_boxes.head()

100%|██████████| 433/433 [00:07<00:00, 56.45it/s]


Bounding Box DataFrame:
(464, 7)


,image_name,x1,y1,x2,y2,confidence,class_id
0,61552862e138234ee400043f,NaN,NaN,NaN,NaN,NaN,NaN
1,6343ce87e138230395000c14,NaN,NaN,NaN,NaN,NaN,NaN
2,5e3a2c5ae1382357cf0004cf,NaN,NaN,NaN,NaN,NaN,NaN
3,6279e78ee138230e060011e3,198.656006,40.943935,231.063782,87.10099,0.501713,3.0
4,62158ac9e1382358fa000068,NaN,NaN,NaN,NaN,NaN,NaN


## Map features with df

In [27]:
train_df = pd.read_csv('/content/liver-fibrosis-severity-prediction/train.csv')
train_df['lesion'] = train_df['image_name'].map(class_dict)

print(train_df.shape)
train_df.head()

(1772, 7)


,subject,image_name,view,SWE fibrosis stage,TE(kPa),TE result,lesion
0,1,5c7e1166e138234adf000b5a,Intercostal,-,3.4,F0,NaN
1,2,5c809dc9e138233fae00064d,Intercostal,-,8.5,F2,NaN
2,3,5cedf3f5e13823798c000355,Subcostal_hepatic_vein,F2,9.4,F3,NaN
3,3,5cedf408e13823798c00035e,Intercostal,F2,9.4,F3,NaN
4,3,5cedf40ce13823798c000364,Liver/RK,F2,9.4,F3,NaN


In [30]:
train_df = train_df.explode('lesion',)
train_df = train_df.reset_index(drop=True)
train_df = train_df.merge(df_boxes, on='image_name', how='left')

print(train_df.shape)

(1891, 13)


In [34]:
test_df = pd.read_csv('/content/liver-fibrosis-severity-prediction/test_submission.csv')
test_df['lesion'] = test_df['image_name'].map(class_dict)
test_df = test_df.explode('lesion',)
test_df = test_df.reset_index(drop=True)
test_df = test_df.merge(df_boxes, on='image_name', how='left')

print(test_df.shape)
test_df.head()

(538, 11)


,image_name,view,SWE fibrosis stage,TE(kPa),lesion,x1,y1,x2,y2,confidence,class_id
0,5c7de4d6e138234adf00044d,Intercostal,-,8.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5d118de2e138237034000368,Liver/RK,F0-1,4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5d1ae0b9e13823585300002f,Intercostal,F0-1,7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5d1ae0c2e138236ca60005b9,Liver/RK,F0-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5d1ae0c4e138236ca60005bd,Subcostal_hepatic_vein,F0-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
train_df.to_csv('train_yolo_added.csv', index=False)
test_df.to_csv('test_yolo_added.csv', index=False)